<a href="https://colab.research.google.com/github/B34R-e/My-Projects/blob/main/Image_Data_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Import necessary libraries**

In [10]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.0PnKxJuIz2/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.MVGg7xsFBC/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.2WC0KmXBiC/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
gpg: cannot open '/dev/tty': No such device or address
gpg: [stdout]: write error: Broken pipe
gpg: filter_flush failed on c

In [11]:
import os
import requests
import time
import random
import selenium

from io import BytesIO
from PIL import Image
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### **Crawl**

In [12]:
# Initialize google chrome browser
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless=new')
chrome_options.add_argument('--no-sandbox')
driver = webdriver.Chrome(
    'chromedriver',
    options = chrome_options
)

# Create a folder for storing articles
root_dir = './vn_news_thumbnail'
os.makedirs(root_dir, exist_ok=True)
n_pages = 10
img_id = 0

for page_idx in tqdm(range(n_pages)):
  # Access to table page
  main_url = f'http://vietnamnet.vn/thoi-su-page{page_idx}'
  driver.get(main_url)

  # Get list of articles (list of URLs)
  imgs_lst_xpath = '//div[@class="topStory-15nd"]/div/div[1]/a/img'
  imgs_tags = driver.find_elements(
      By.XPATH,
      imgs_lst_xpath
  )
  img_urls = [imgs_tag.get_attribute('src') for imgs_tag in imgs_tags]

  # Downloading and save image
  for img_url in img_urls:
    img_url_resp = requests.get(img_url)
    try:
      img = Image.open(
          BytesIO(img_url_resp.content)
      )
    except:
      continue

    if img.mode == 'P':
      img = img.convert('RGB')
    
    img_name = f'IMG_{img_id:05}.png'
    img_save_path = os.path.join(
        root_dir,
        img_name
    )
    img.save(img_save_path)
    img_id += 1

100%|██████████| 10/10 [03:31<00:00, 21.11s/it]


In [13]:
!zip -r vn_news_thumbnail.zip vn_news_thumbnail

updating: vn_news_thumbnail/ (stored 0%)
  adding: vn_news_thumbnail/IMG_00020.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00114.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00115.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00127.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00062.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00046.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00014.png (deflated 1%)
  adding: vn_news_thumbnail/IMG_00003.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00068.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00120.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00052.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00034.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00145.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00142.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00078.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00088.png (deflated 0%)
  adding: vn_news_thumbnail/IMG_00118.png (deflated 0%)
  addin